In [1]:
import pickle
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [2]:
kwid = pd.read_pickle('kwid.pickle')
kwdata = pd.read_pickle('synonymsdata.pickle')
matrix = pd.read_pickle('largematrix.pickle')
p = pd.read_pickle('p.pickle')

In [3]:
for element in p:
    kwdata[element[0]]['importance'] = element[1]

In [4]:
def out1(a):
    return a[1]

def reorderconditional(searchid, matrix):
    temp = []
    for i in range(len(matrix[searchid])):
        if i!=searchid:
            temp.append([i, matrix[searchid, i]])
    temp = sorted(temp, key=out1, reverse=True)
    return temp

# 80-20% principle
def gettopconnections(searchid, matrix, kwdata=None):
    print(f"importnce of {kwdata[searchid]['name']}:{kwdata[searchid]['importance']}")
    totalsum = sum(sum(matrix))
    returnthis = []
    sigma = sum(matrix[searchid])
    temp = reorderconditional(searchid, matrix)
    tempsum = 0
    i = 0
    while True:
        returnthis.append(temp[i][0])
        tempsum+=temp[i][1]
        P = sum(matrix[:, temp[i][0]])/totalsum
        p = matrix[searchid, temp[i][0]]/sum(matrix[searchid])
        if kwdata is not None:
            print(f"Importance:{kwdata[temp[i][0]]['importance']}, P({kwdata[temp[i][0]]['name']}):{P}, P({kwdata[temp[i][0]]['name']}|{kwdata[searchid]['name']}):{p}")
        if tempsum>0.2*sigma:
            break
        i+=1
    return returnthis

In [5]:
gettopconnections(kwid['bitcoin'], matrix, kwdata)

importnce of bitcoin:0.03677962519374384
Importance:0.08157319994363815, P(crypto):0.07166260991849793, P(crypto|bitcoin):0.1059753997911352
Importance:0.1859518106242074, P(blockchain):0.2116505567452529, P(blockchain|bitcoin):0.09369735377262836
Importance:0.00731647174862618, P(mining):0.008956243197811641, P(mining|bitcoin):0.023040162263817464


[139, 57, 133]

In [6]:
gettopconnections(kwid['pow'], matrix, kwdata)

importnce of proof of work:0.007799070029589968
Importance:0.1859518106242074, P(blockchain):0.2116505567452529, P(blockchain|proof of work):0.12771588847108684
Importance:0.019120755248696632, P(consensus):0.018723049973257435, P(consensus|proof of work):0.10204011123962595


[57, 111]

In [7]:
matrix[kwid['consensus'], kwid['consensus']]/sum(sum(matrix))

0.005740825576478769

In [8]:
### updated functions

def reorderconditional(searchid, matrix, transpose=False):
    temp = []
    if transpose==False:
        for i in range(len(matrix[searchid])):
            if i!=searchid:
                temp.append([i, matrix[searchid, i]])
        temp = sorted(temp, key=out1, reverse=True)
    elif transpose==True:
        for i in range(len(matrix[:, searchid])):
            if i!=searchid:
                temp.append([i, matrix[i, searchid]])
        temp = sorted(temp, key=out1, reverse=True)
    return temp

def getprereqs(searchid, matrix=matrix, kwdata=kwdata):
    #
    returnthis = []
    #
    psearchterm = kwdata[searchid]['importance']
    sigma = sum(matrix[searchid])
    conditionalorder = reorderconditional(searchid, matrix)
    #
    tempsum = 0
    i = 0
    #
    for element in conditionalorder:
        tempsum+=element[1]
        if kwdata[element[0]]['importance']>psearchterm:
            returnthis.append(kwdata[element[0]]['name'])
        if tempsum>0.2*sigma:
            break
    #
    return returnthis

def getroadmap(searchid, matrix=matrix, kwdata=kwdata):
    #
    returnthis = []
    #
    psearchterm = kwdata[searchid]['importance']
    sigma = sum(matrix[:, searchid])
    conditionalorder = reorderconditional(searchid, matrix, transpose=True)
    #
    tempsum = 0
    i = 0
    #
    for element in conditionalorder:
        tempsum+=element[1]
        if kwdata[element[0]]['importance']<psearchterm:
            returnthis.append(kwdata[element[0]]['name'])
        if tempsum>0.2*sigma:
            break
    return returnthis

def search(searchid):
    prereqs = getprereqs(searchid)
    roadmap = getroadmap(searchid)
    return {'prereqs':prereqs, 'roadmap':roadmap}

In [10]:
search(kwid['blockchain'])

{'prereqs': [],
 'roadmap': ['crypto',
  'decentralized',
  'token',
  'smart contracts',
  'ethereum',
  'hyperledger',
  'bitcoin',
  'consensus',
  'nfts',
  'healthcare',
  'distributed ledger',
  'protocol',
  'iot',
  'scalability',
  'ico',
  'interoperability',
  'dapps']}

In [11]:
def testname(a, kwdata=kwdata, kwid=kwid):
    return kwdata[kwid[a]]['importance']

sorted(search(kwid['blockchain'])['roadmap'], key=testname, reverse=True)

['crypto',
 'token',
 'bitcoin',
 'ethereum',
 'decentralized',
 'smart contracts',
 'nfts',
 'protocol',
 'consensus',
 'ico',
 'dapps',
 'hyperledger',
 'scalability',
 'iot',
 'distributed ledger',
 'healthcare',
 'interoperability']